In [1]:
import requests
import json
from Crypto.PublicKey import RSA
import time
import datetime

from bs4 import BeautifulSoup

import getpass

from util.crypto import encrypt_data
from util.html_tools import get_variable_from_html, get_json_variable_from_html

In [2]:
def create_steam_auth_session():
    session = requests.Session()
    response = session.get("https://steamcommunity.com/login/home/?goto=")
    user = input("Steam Username:")
    password = getpass.getpass("Password:")
    login_params = {
        "username": user
    }

    response = session.post("https://steamcommunity.com/login/home/getrsakey/",
        params = login_params)
    responseJSON = json.loads(response.text)
    
    
    exp = int(responseJSON["publickey_exp"],16)
    mod = int(responseJSON["publickey_mod"],16)
    
    rsa_key = RSA.construct((mod, exp))
    
    encrypted_password = encrypt_data(password, rsa_key)
    
    twofactorcode = ""
    m_gidCaptcha = "-1"
    captchaText = ""
    m_steamidEmailAuth = ""
    m_unRequestedTokenType = "-1"
    login_params = {
        "donotcache":round(time.time() * 1000),
        "password":encrypted_password,
        "username":user,
        "twofactorcode": twofactorcode,
        "emailauth":"",
        "loginfriendlyname":"",
        "captchagid":m_gidCaptcha,
        "captcha_text":captchaText,
        "emailsteamid":m_steamidEmailAuth,
        "rsatimestamp":responseJSON["timestamp"],
        "remember_login": 'false',
        "tokentype": m_unRequestedTokenType
        
    }
    
    header = {
        "Content-Type":"application/x-www-form-urlencoded; charset=UTF-8",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }
    
    
    response = session.post("https://steamcommunity.com/login/home/dologin",
                     data = login_params
                     )
    responseJSON = json.loads(response.text)

    if responseJSON["success"] == True:
        print(f"Successfully logged in {user}")
        return s

    require_twofactor = False
    require_email = False
    if "requires_twofactor" in responseJSON:
        if responseJSON["requires_twofactor"] == True:
            require_twofactor = True
    if "emailauth_needed" in responseJSON:
        if responseJSON["emailauth_needed"] == True:
            require_email = True

    if not require_email and not require_twofactor:
        print("Session could not be established, wrong password or username")
        return None
    
    if require_twofactor:
        twofactorcode = input("Enter your 2FA code:")
        login_params["twofactorcode"] = twofactorcode
    else:
        email_code = input("Enter your email confirmation code:")
        login_params["emailauth"] = email_code

    response = session.post("https://steamcommunity.com/login/home/dologin",
                 data = login_params
                 )
    responseJSON = json.loads(response.text)
    if responseJSON["success"] == True:
        return session
    else:
        print("Login failed")
        print(responseJSON)
    
        

In [2]:
def create_inventory_history_list(html_string, items_json):
    soup = BeautifulSoup(html_string, 'html.parser')
    history_list = []
    rows = soup.find_all("div", "tradehistoryrow")
    for row in rows:
        history_element = {
            "description": "",
            "timestamp": None,
            "new_items": [],
            "lost_items": [],
        }
        date = row.find_all("div", "tradehistory_date")[0].text.strip().split("\t")
        date = [x for x in date if x]
        day = int(date[0].split(" ")[0])
        month = date[0].split(" ")[1][:-1]
        year = int(date[0].split(" ")[-1])
        hour = int(date[1].split(":")[0])
        minute = int(date[1].split(":")[1][:-2])
        ampm = date[1][-2:]
        history_element["timestamp"] = time.mktime(datetime.datetime.strptime(f"{day:02d}/{month}/{year} {hour:02d}:{minute:02d} {ampm}", "%d/%b/%Y %H:%M %p").timetuple())
        history_element["description"] = row.find_all("div", "tradehistory_event_description")[0].text.strip()
        item_changes = row.find_all("div", "tradehistory_items tradehistory_items_withimages")
        for item_change in item_changes:
            item_type = item_change.find("div", "tradehistory_plusminus").text.strip()
            print(item_type)
            print(item_change)
        
        history_list.append(history_element)
    pass

In [3]:
def get_inventory_history(session):
    response = session.get("https://steamcommunity.com/")
    if response.status_code != 200:
        return None
    steamid = get_variable_from_html("g_steamID", response.text)
    response = session.get(f"https://steamcommunity.com/profiles/{steamid}/inventoryhistory/?app[]=730")
    if response.status_code != 200:
        return None
    profile_link = get_variable_from_html("g_strProfileURL", response.text).replace("\\", "")
    sessionid = get_variable_from_html("g_sessionID", response.text)
    filter_apps = get_variable_from_html("g_rgFilterApps", response.text)


    cursor = get_variable_from_html("g_historyCursor", response.text)
    item_json = get_json_variable_from_html("g_rgDescriptions", response.text)

    full_site = response.text
    inventory_history = create_inventory_history_list(full_site, item_json)

    while True:
        return
        req_data = {
            "ajax": "1",
            "sessionid": sessionid,
        }
        for key in cursor:
            req_data[f"cursor[{key}]"] = cursor[key]
        for num, app in enumerate(filter_apps):
            req_data[f"app[{num}]"] = app
        response = session.get(profile_link+"/inventoryhistory/", params=req_data)
        response_JSON = json.loads(response.text)
        if cursor in response_JSON.keys():
            cursor = response_JSON["cursor"]
        else:
            break  
    return full_site

In [5]:
session = create_steam_auth_session()

In [6]:
soup, test = get_inventory_history(session)

In [8]:
json.loads(test)["730"]

{'4693725102_4834422536': {'icon_url': '-9a81dlWLwJ2UUGcVs_nsVtzdOEdtWwKGZZLQHTxDZ7I56KU0Zwwo4NUX4oFJZEHLbXQ9Q1LO5kNoBhSQl-fVOG_wcbQVmJ5IABWuoX3e1Uy7P_efWwMuI63ldCOlKWlZOLQxTwAucQjj-yR8d-m2VfhqkBvZGmlIoXBIVJoM0aQpAZRtkzP5w',
  'icon_url_large': '-9a81dlWLwJ2UUGcVs_nsVtzdOEdtWwKGZZLQHTxDZ7I56KU0Zwwo4NUX4oFJZEHLbXQ9Q1LO5kNoBhSQl-fVOG_wcbQVmJ5IABWuoX3e1Uy7P_efWx94N2kk4XFlKKjZuyAkDpUv5Mh2e_E9tig2VG3rRFta2r3I4STJ1I4aVjWqVa3wvCv28E5vXoJEg',
  'icon_drag_url': '',
  'name': '2022 Service Medal',
  'market_hash_name': '2022 Service Medal',
  'market_name': '2022 Service Medal',
  'name_color': 'D2D2D2',
  'background_color': '',
  'type': 'Extraordinary Collectible',
  'tradable': 0,
  'marketable': 0,
  'commodity': 0,
  'market_tradable_restriction': '7',
  'descriptions': [{'type': 'html',
    'value': 'Awarded for outstanding service and achievement in 2022.'},
   {'type': 'html', 'value': ' '},
   {'type': 'html',
    'value': 'Date of Issue: May 06, 2022 GMT',
    'color': '99ccff'},
   

In [24]:
soup.find_all("script")

[<script type="text/javascript">
 	var __PrototypePreserve=[];
 	__PrototypePreserve[0] = Array.from;
 	__PrototypePreserve[1] = Function.prototype.bind;
 	__PrototypePreserve[2] = HTMLElement.prototype.scrollTo;
 </script>,
 <script src="https://community.akamai.steamstatic.com/public/javascript/prototype-1.7.js?v=.55t44gwuwgvw" type="text/javascript"></script>,
 <script type="text/javascript">
 	Array.from = __PrototypePreserve[0] || Array.from;
 	Function.prototype.bind = __PrototypePreserve[1] || Function.prototype.bind;
 	HTMLElement.prototype.scrollTo = __PrototypePreserve[2] || HTMLElement.prototype.scrollTo;
 </script>,
 <script type="text/javascript">VALVE_PUBLIC_PATH = "https:\/\/community.akamai.steamstatic.com\/public\/";</script>,
 <script src="https://community.akamai.steamstatic.com/public/javascript/scriptaculous/_combined.js?v=OeNIgrpEF8tL&amp;l=english&amp;load=effects,controls,slider,dragdrop" type="text/javascript"></script>,
 <script src="https://community.akamai.s